In [1]:
#Import library
import requests as re
import pandas as pd
import requests
import numpy as np
import pandas as pd
import json
from datetime import datetime
import matplotlib.pyplot as plt
import time
import os
from pathlib import Path

#### API exploration
The API is limit to 1000 output in each return. Hence we need to use offset attribute in the API query to get the next 1000 rows of data

In [6]:
## Example of the API query
Token = '<your_noaa_token>'
offset=0
req_url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/locations?locationcategoryid=CITY&limit=1000&offset={}'.format(offset)
locations = re.get(req_url, headers={'token':Token})
d = json.loads(locations.text)
#Show top 5 results
d['results'][0:5]

[{'mindate': '1983-01-01',
  'maxdate': '2021-07-29',
  'name': 'Abu Dhabi, AE',
  'datacoverage': 0.9977,
  'id': 'CITY:AE000001'},
 {'mindate': '1944-03-01',
  'maxdate': '2021-07-29',
  'name': 'Ajman, AE',
  'datacoverage': 0.9991,
  'id': 'CITY:AE000002'},
 {'mindate': '1944-03-01',
  'maxdate': '2021-07-29',
  'name': 'Dubai, AE',
  'datacoverage': 0.9991,
  'id': 'CITY:AE000003'},
 {'mindate': '1944-03-01',
  'maxdate': '2021-07-29',
  'name': 'Sharjah, AE',
  'datacoverage': 0.9991,
  'id': 'CITY:AE000006'},
 {'mindate': '1966-03-02',
  'maxdate': '2021-07-29',
  'name': 'Kabul, AF',
  'datacoverage': 0.9969,
  'id': 'CITY:AF000007'}]

In [5]:
locations

<Response [404]>

|date|location_id|city_name|TAVG|station_id|long|lat

#### method to be developed and test
1. GetAllCityCode - make an API query to get the list of cities in 

In [24]:
def GetAllCityCode(Token):
    FullResult=[]
    for i in range(100):
        offset=i*1000
        req_url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/locations?locationcategoryid=CITY&limit=1000&offset={}'.format(offset)
        locations = re.get(req_url, headers={'token':Token})
        d = json.loads(locations.text)
        print(d)
        if len(d)==0:
            print("empty string exit",i)
            break
        FullResult.extend(d['results'])
    #print(FullResult)
    return pd.DataFrame(FullResult).drop_duplicates()
        

In [8]:
def GetLongLat(inToken,station_id):
    """
    arg: inToken,stationid
    return long lat 
    """
    #print(station_id)
    req_url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/stations/'+station_id
    longlat = re.get(req_url, headers={'token':inToken})
    if len(longlat.text)==2:  #2 character {}
        return 0,0
    json_longlat=json.loads(longlat.text)
    return json_longlat['longitude'],json_longlat['latitude']

In [9]:
def GetPrecipitation(inToken,station_id,start_date,end_date):
    """
    arg: inToken,stationid
    return df with the data 
    """
    req_url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000' \
        '&startdate='+start_date+'&enddate='+end_date+'&stationid='+station_id
    #print(req_url)
    prcp = re.get(req_url, headers={'token':inToken})
    if len(prcp.text)==2:  #2 character {}
        return pd.DataFrame(columns = ['attributes','datatype','date','station','prcp'])
    #print(prcp.text)
    json_prcp = json.loads(prcp.text)
    df_prcp = pd.DataFrame(json_prcp['results'])
    df_prcp=df_prcp.rename(columns={"value":"prcp"})
    return df_prcp

In [14]:
def GetTemperature(inToken,location_id,city_name,start_date,end_date):
    req_url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&locationid='+location_id+ \
            '&startdate='+start_date+'&enddate='+end_date
    weather = re.get(req_url, headers={'token':inToken})
    time.sleep(0.2) #avoid next API request got cancelled due to hit the 5 req per sec quota
    #print(len(weather.text))
    if len(weather.text)==2:  #2 character {}
        print(len(weather.text))
        return None
    json_weather = json.loads(weather.text)
    df_weather = pd.DataFrame(json_weather['results'])
    df_weather['location_id']=location_id
    df_weather['city_name']=city_name
    df_weather['average_temp']=df_weather['value'] / 10
    station_unique = df_weather['station'].unique() #apply to each row is better as big city has multiple read
    print(location_id,city_name,station_unique)
    #get long lat
    long_lats = [GetLongLat(inToken,i) for i in station_unique]
    df_stn_long_lat=pd.DataFrame({'station_id':station_unique,'long_lat':long_lats})
    df_stn_long_lat[['long', 'lat']] = pd.DataFrame(df_stn_long_lat['long_lat'].tolist(), index=df_stn_long_lat.index)
    df_stn_long_lat=df_stn_long_lat.drop(['long_lat'],axis=1)
    df_weather = df_weather.merge(df_stn_long_lat,  how='left', left_on='station', right_on='station_id')
    #precipitation
    list_prcp = [GetPrecipitation(inToken,i,start_date,end_date) for i in station_unique]
    #print(list_prcp)
    df_prcp = pd.concat(list_prcp)
    df_weather = df_weather.merge(df_prcp,  how='left', left_on=['station','date'], right_on = ['station','date'])
    #print(df_prcp.head())
    #subset the data
    df_weather=df_weather[['date','location_id','city_name','average_temp','station','long','lat','prcp']]
    return df_weather

In [25]:
Token = 'pMnUqnCVOhzBBBgneXwexkNCSaxdafuL'
df = GetAllCityCode(Token)
df=df[df['name'].str.contains("US")][0:5]

{'status': '429', 'message': 'This token has reached its temporary request limit of 10000 per day.'}


KeyError: 'results'

In [12]:
#Exploring finding the US city that contains "Ph"
#df[df['name'].str.contains("US") &df['name'].str.contains("Ph")]

,datacoverage,id,maxdate,mindate,name
1106,1.0,CITY:US040011,2021-08-01,1893-01-01,"Phoenix, AZ US"
1650,1.0,CITY:US390019,2021-08-01,1893-06-01,"New Philadelphia, OH US"
1712,1.0,CITY:US420015,2021-08-01,1876-12-24,"Philadelphia, PA US"


In [16]:
#test the method in getting the data
start_time=time.time()
full_df = pd.concat([GetTemperature(Token,x, y,'2021-06-01','2021-06-30') for x, y in zip(df['id'], df['name'])])
print(time.time()-start_time)

CITY:US000001 Washington D.C., US ['GHCND:USW00013743' 'GHCND:USW00093721' 'GHCND:USW00093738']
2
CITY:US010002 Anniston, AL US ['GHCND:USR0000ASHK']
2
CITY:US010004 Birmingham, AL US ['GHCND:USW00013876']
6.668336629867554


In [22]:
GetPrecipitation(Token,'GHCND:USW00093738','2021-07-23','2021-07-23')

,attributes,datatype,date,station,prcp
0,",,D,2400",PRCP,2021-07-23T00:00:00,GHCND:USW00093738,0


In [13]:
full_df.head()

,date,location_id,city_name,average_temp,station,long,lat,prcp
0,2021-06-01T00:00:00,CITY:US000001,"Washington D.C., US",19.8,GHCND:USW00013743,-77.03454,38.8472,0
1,2021-06-01T00:00:00,CITY:US000001,"Washington D.C., US",18.8,GHCND:USW00093721,-76.68400,39.1733,0
2,2021-06-01T00:00:00,CITY:US000001,"Washington D.C., US",18.3,GHCND:USW00093738,-77.44730,38.9349,0
3,2021-06-02T00:00:00,CITY:US000001,"Washington D.C., US",21.9,GHCND:USW00013743,-77.03454,38.8472,0
4,2021-06-02T00:00:00,CITY:US000001,"Washington D.C., US",21.2,GHCND:USW00093721,-76.68400,39.1733,0


In [8]:
Token = 'pMnUqnCVOhzBBBgneXwexkNCSaxdafuL'
location_id='CITY:US000001'
city_name='Washington D.C., US'
start_date='2021-06-01'
end_date='2021-06-03'
inToken=Token
req_url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&locationid='+location_id+ \
        '&startdate='+start_date+'&enddate='+end_date
weather = re.get(req_url, headers={'token':inToken})
#print(len(weather.text))
#if len(weather.text)==2:  #2 character {}
#    return None
json_weather = json.loads(weather.text)
df_weather = pd.DataFrame(json_weather['results'])
df_weather['location_id']=location_id
df_weather['city_name']=city_name
df_weather['average_temp']=df_weather['value'] / 10


In [ ]:
df_weather.merge(df_prcp,  how='left', left_on=['station','date'], right_on = ['station','date'])

In [19]:
df_prcp.sort_values(by=['station','date'])

,attributes,datatype,date,station,value
0,",,D,2400",PRCP,2021-06-01T00:00:00,GHCND:USW00013743,0
1,",,D,2400",PRCP,2021-06-02T00:00:00,GHCND:USW00013743,0
2,",,D,2400",PRCP,2021-06-03T00:00:00,GHCND:USW00013743,25


In [ ]:
#https://www.ncdc.noaa.gov/cdo-web/api/v2/datatypes?datacategoryid=TEMP&limit=56
req_url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/datatypes?datacategoryid=PRCP&limit=56'
print(req_url)
weather = re.get(req_url, headers={'token':inToken})
json_d = json.loads(weather.text)
json_d

In [152]:
station_unique = df_weather['station'].unique() #apply to each row is better as big city has multiple read
long_lats = [GetLongLat(inToken,i) for i in station_unique]
df_stn_long_lat=pd.DataFrame({'station_id':station_unique,'long_lat':long_lats})
df_stn_long_lat[['long', 'lat']] = pd.DataFrame(df_stn_long_lat['long_lat'].tolist(), index=df_stn_long_lat.index)
df_stn_long_lat=df_stn_long_lat.drop(['long_lat'],axis=1)
df_weather = df_weather.merge(df_stn_long_lat, left_on='station', right_on='station_id')

GHCND:USW00013743
GHCND:USW00093721
GHCND:USW00093738


In [ ]:
#long,lat = GetLongLat(inToken,station_unique)
df_weather = df_weather[['date','location_id','city_name','average_temp','station']]
df_weather[['long','lat']]=df_weather.apply(lambda x: GetLongLat(inToken,x.station),axis=1)
#df_weather['long']=long
#df_weather['lat']=lat

In [ ]:
90.5, 93, 93.5

In [191]:
325,339,441

3.679144385026738

#### Testing the full script

In [11]:
full_df.to_csv('data.csv')

In [13]:
#testing the full method
%load_ext autoreload
%autoreload
import NOAA_temperature_data as noaa

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:

full_data = noaa.acquire_temperature_data('2021-07-08','2021-07-10')

empty string exit
CITY:US000001 Washington D.C., US ['GHCND:USW00013743' 'GHCND:USW00093721' 'GHCND:USW00093738']
2
CITY:US010002 Anniston, AL US ['GHCND:USR0000ASHK']
2
CITY:US010004 Birmingham, AL US ['GHCND:USW00013876']
5.087112903594971


In [40]:
#testing to get unique date
ts = pd.to_datetime(full_data['date'], format='%Y-%m-%dT%H:%M:%S')#.dt.strftime('%Y-%m-%d')
ts.unique()

0   2021-07-08
1   2021-07-08
2   2021-07-08
0   2021-07-08
0   2021-07-08
Name: date, dtype: datetime64[ns]

In [23]:
#testing the script to save file as csv
for i, x in full_data.groupby(ts):
    outdir = '{}/{}/{}'.format(i.year,i.month,i.day)
    if not os.path.exists(outdir):
        os.makedirs(outdir, exist_ok=True)
    x.to_csv(outdir+'/temperature_data.csv', index=False)